# Unconfined aquifer pumping test


A solution to the steady-state drawdown (cone of depression) in an unconfined aquifer was derived by Theim - the "Theim solution". Consider the diagram below. $b$ represents the saturated thickness. Prior to pumping, the water table is at the static water level, and the saturated thickness is $b_0$ (m). We can work out $b_0$ if we know the depth to the base of the aquifer (in our case the base of the sand) from drillers logs, and we know the depth below ground of the water table before pumping.

When the aquifer is pumped for some period of time, with the pump still running, we should check whether or not the water table has stopped changing. Once we are convinced that the water table is not changing, we can be satisfied that we have reached steady conditions, and that this Theim method of analysis is appropriate.

![](Fetter_Theim.png)

Figure 1. Steady-state pumping cone of depression (from Fetter, Applied Hydrogeology, 4th edition, p.167)

The Thiem solution to calculate the saturated thickness of the aquifer, $b$ (m), at a given radius, $r$ (m), is given by

$$b^2-b_1^2=\frac{Q}{\pi K}\ln\left(\frac{r}{r_1}\right)$$

where $K$ is hydraulic conductivity (m/min), $Q$ is pumping rate (m$^3$/min), $b_1$ is a known saturated thickness at distance $r_1$ (which could be one known point on the cone of depression). Changing $K$ changes the steepness of the cone of depression. The above equation can be used to simulate the aquifer thickness, or the equivalent drawdown, and compared with observed drawdown values. 

This notebook is set up to load drawdown data for different piezometers and to compare the Theim solution with the observations. However, to make this work, you will need to enter three parameters, where indicated below. Do not change anything else in this script except to replace the `???`.


In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd

In [ ]:
# The Theim solution: calculates the saturated thickness as a function of radius 
# for a given flow rate and hydraulic conductivity
def Theim(r,r1,b1,Q,K):
    LHS=Q/np.pi/K*np.log(r/r1)
    b=np.sqrt(LHS+b1**2)
    return b

In [ ]:
# Load ground elevation and piezometer distances from well
GE,r=np.loadtxt('WellPiezoData.csv',delimiter=',',skiprows=1,usecols=(1,2),unpack=True)

In [ ]:
# Load drawdown data
t,h4d=np.loadtxt('Piez4_drawdown.csv',delimiter=',',skiprows=1,unpack=True)
t,h3d=np.loadtxt('Piez3_drawdown.csv',delimiter=',',skiprows=1,unpack=True)
t,h1d=np.loadtxt('Piez1_drawdown.csv',delimiter=',',skiprows=1,unpack=True)
t,wd=np.loadtxt('Well_drawdown.csv',delimiter=',',skiprows=1,unpack=True)

In [ ]:
# Convert to drawdown and put into an array ddn(radius,time):
h4d=h4d-h4d[0]
h3d=h3d-h3d[0]
h1d=h1d-h1d[0]
wd=wd-wd[0]
ddn=np.array([wd,h1d,h3d,h4d])
rad=r[[0,1,3,4]]

In [ ]:
pl.figure(figsize=(10,4))
pl.subplot(1,2,1)
pl.plot(rad,ddn[:,:],'.-')
pl.ylim(1,-.05)
pl.grid()
pl.xlabel('Distance from well (m)')
pl.ylabel('Drawdown (m)')
pl.title('All time (t=0 to 60 min)')
pl.subplot(1,2,2)
pl.plot(rad,ddn[:,-3:],'.-')
pl.ylim(1,-.05)
pl.grid()
pl.xlabel('Distance from well (m)')
pl.title('Late time (t>30 min)')
pl.show()

### Instructions:

In the block of code below you must enter the pumping rate, `Q`, the resting saturated thickness, `b0`, and the hydraulic conductivity, `K`. You should find the best `K` value that fits the observed drawdown.  

In [ ]:
# Parameters
K=???        # m/min
Q=???        # m3/min
b0=???       # m

# Do not change the code below here:
# Take piezo 1 as a known condition, r1,b1:
r1=r[1]
b1=b0-ddn[1,-1]

# Solve and plot against observations:
ra=np.linspace(0,30)
ra[0]=0.005
b=Theim(ra,r1,b1,Q,K)
pl.plot(ra,b,'r',label='Theim solution')
pl.plot(rad,b0-ddn[:,-1],'xk',label='Observations (t$\geq$30 min)')
pl.plot(rad,b0-ddn[:,-3:],'xk')
pl.xlabel('Distance from well (m)',fontsize=12)
pl.ylabel('Drawdown (m)',fontsize=12)
pl.title('Fitting performance',fontsize=12)
pl.legend(loc=4,fontsize=12)
pl.show()

In [ ]:
print('The hydraulic conductivity of the aquifer is %.1e m/min'%(K/60))
print('The hydraulic conductivity of the aquifer is %.3f m/d'%(K*60*24))

In [ ]:
# Nice plot
pl.figure(figsize=(13,4))
pl.subplots_adjust(wspace=0.02)
pl.subplot(1,3,1)
pl.plot(rad,ddn[:,:],'.-')
pl.ylim(1,-.05)
pl.grid()
pl.xlabel('Distance from well (m)',fontsize=12)
pl.ylabel('Drawdown (m)',fontsize=12)
pl.title('All time (t=0 to 60 min)',fontsize=12)

pl.subplot(1,3,2)
pl.plot(rad,ddn[:,-3:],'.-')
pl.ylim(1,-.05)
pl.grid()
pl.xlabel('Distance from well (m)',fontsize=12)
pl.title('Late time (t>30 min)',fontsize=12)
pl.gca().set_yticklabels('')

pl.subplot(1,3,3)
pl.plot(rad,ddn[:,-1],'xk',label='Observations (t$\geq$30 min)')
pl.plot(rad,ddn[:,-3:],'xk')
pl.plot(ra,b0-b,'r',linewidth=2,label='Theim solution, K= %.0f m/d'%(K*24*60))
pl.legend(loc=4,fontsize=12)
pl.ylim(1,-.05)
pl.grid()
pl.xlabel('Distance from well (m)',fontsize=12)
pl.title('Theim solution for late time',fontsize=12)
pl.gca().set_yticklabels('')

# pl.show()

pl.savefig('Drawdown.png')